In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sb

In [3]:
# hip = pd.read_csv("CompleteData/hip_complete.csv", index_col = 0)
# wrist = pd.read_csv("CompleteData/wrist_complete.csv", index_col = 0)
# biocomplete = pd.read_csv("CompleteData/bio_complete.csv", index_col = 0)
# thigh = biocomplete.drop([col for col in biocomplete.columns if "chest" in col], axis = 1)
# chest = biocomplete.drop([col for col in biocomplete.columns if "thigh" in col], axis = 1)
# thigh_cols = [col[6:] if "thigh" in col else col for col in thigh.columns ]
# thigh.columns = thigh_cols
# chest_cols = [col[6:] if "chest" in col else col for col in chest.columns ]
# chest.columns = chest_cols

In [6]:
hip = pd.read_csv("Final_Data/Hip.csv")
wrist = pd.read_csv("Final_Data/Wrist.csv")
thigh = pd.read_csv("Final_Data/Thigh.csv")
chest = pd.read_csv("Final_Data/Chest.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
hip.columns

Index(['Unnamed: 0', 'observation', 'date', 'coding', 'primary_behavior',
       'primary_posture', 'primary_upperbody', 'primary_intensity',
       'secondary_behavior', 'secondary_posture',
       ...
       'mfft12', 'mfft13', 'mfft14', 'mfft15', 'broad_activity',
       'detailed_activity', 'walking_running_bouts', 'updated_activity',
       'final_activity', 'updated_final_activity'],
      dtype='object', length=108)

In [9]:
wrist.columns

Index(['Unnamed: 0', 'observation', 'date', 'coding', 'primary_behavior',
       'primary_posture', 'primary_upperbody', 'primary_intensity',
       'secondary_behavior', 'secondary_posture',
       ...
       'mfft12', 'mfft13', 'mfft14', 'mfft15', 'broad_activity',
       'detailed_activity', 'walking_running_bouts', 'updated_activity',
       'final_activity', 'updated_final_activity'],
      dtype='object', length=108)

In [10]:
thigh.columns

Index(['Unnamed: 0', 'observation', 'date', 'coding', 'primary_behavior',
       'primary_posture', 'primary_upperbody', 'primary_intensity',
       'secondary_behavior', 'secondary_posture',
       ...
       'mfft12', 'mfft13', 'mfft14', 'mfft15', 'broad_activity',
       'detailed_activity', 'walking_running_bouts', 'updated_activity',
       'final_activity', 'updated_final_activity'],
      dtype='object', length=108)

In [11]:
chest.columns

Index(['Unnamed: 0', 'observation', 'date', 'coding', 'primary_behavior',
       'primary_posture', 'primary_upperbody', 'primary_intensity',
       'secondary_behavior', 'secondary_posture',
       ...
       'mfft12', 'mfft13', 'mfft14', 'mfft15', 'broad_activity',
       'detailed_activity', 'walking_running_bouts', 'updated_activity',
       'final_activity', 'updated_final_activity'],
      dtype='object', length=108)

# Broad Activity Encoding

Description: creates a new column in the dataset called "broad_activity" with the broad activity encodings

data - dataset (hip, wrist, thigh, or chest)

In [12]:
def get_broad_activity_type(data):
    broad = []
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        if row["primary_posture"] == "private/not coded":
            broad.append("private/not coded")
        elif row["coding"] == "sedentary":
            if row["primary_behavior"] in ["TRAV- driver (car/truck/motorcycle)", "TRAV- passenger bus or train"]:
                broad.append("vehicle")
            else:
                broad.append("sit/stand")
        else:
            if row["primary_posture"] in ["WA- ascend stairs", "WA- descend stairs",
                                   "WA- running", "WA- walk", "WA-walk with load"]:
                broad.append("walking")
            elif row["primary_posture"] == "LA- stand":
                broad.append("sit/stand")
            elif row["primary_posture"] in ["LA- stand", "LA- stand and move",
                                            "LA- stand and move with unidentifiable upper body movement",
                                            "LA- stand and move with upper body movement",
                                            "SP- other sport movement"]:
                broad.append("mixed-activity")
            elif row["primary_posture"] == "SP- bike":
                broad.append("bicycling")
            else:
                print("BAD")
    data["broad_activity"] = broad
    return data

# Detailed Actvity Encoding

Description: creates a new column in the dataset called “detailed_activity” (does not correspond to detailed activity in our final paper) with the detailed activity encodings.

data - dataset (hip, wrist, thigh, or chest)

In [13]:
def get_detailed_activity_type(data):
    detail = []
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        if row["primary_posture"] == "private/not coded":
            detail.append("private/not coded")
        elif row["coding"] == "sedentary":
            if row["primary_behavior"] in ["TRAV- driver (car/truck/motorcycle)", "TRAV- passenger bus or train"]:
                detail.append("vehicle")
            else:
                detail.append("sit/lie")
        else:
            if row["primary_posture"] in ["WA- ascend stairs", "WA- descend stairs",
                                   "WA- walk", "WA-walk with load"]:
                if row["primary_intensity"] == "light":
                    detail.append("walking light")
                elif row["primary_intensity"] in ["moderate", "vigorous"]:
                    detail.append("walking moderate or vigorous")
            elif row["primary_posture"] == "WA- running":
                detail.append("running")
            elif row["primary_posture"] == "LA- stand":
                detail.append("stand still")
            elif row["primary_posture"] in ["LA- stand and move",
                                            "LA- stand and move with unidentifiable upper body movement",
                                            "LA- stand and move with upper body movement",
                                            "SP- other sport movement"]:
                if row["primary_behavior"] in ['HA- animals and pets',
                                                'HA- exterior maintenance, repair, & decoration',
                                                'HA- food prep and cleanup',
                                                'HA- household management/other household activities',
                                                'HA- housework', 'HA- interior maintenance, repair, & decoration',
                                                'HA- lawn, garden and houseplants']:
                    detail.append("housework")
                elif row["primary_behavior"] == "EX- jogging":
                    if row["primary_intensity"] == "light":
                        detail.append("walking light")
                    elif row["primary_intensity"] in ["moderate", "vigorous"]:
                        detail.append("walking moderate or vigorous")
                elif row["primary_behavior"] == "EX- walking" and row["primary_posture"] not in ["LA- stand and move",
                                                                                                "LA- stand and move with unidentifiable upper body movement",
                                                                                                "LA- stand and move with upper body movement"]:
                    if row["primary_intensity"] == "light":
                        detail.append("walking light")
                    elif row["primary_intensity"] in ["moderate", "vigorous"]:
                        detail.append("walking moderate or vigorous")
                elif row["primary_intensity"] == "light":
                    detail.append("stand and move light")
                elif row["primary_intensity"] in ["moderate", "vigorous"]:
                    detail.append("stand and move moderate or vigorous")
            elif row["primary_posture"] == "SP- bike":
                detail.append("bicycling")
    data["detailed_activity"] = detail
    return data

# Walking or Running Bouts

Description: creates a new column in the dataset called “walking_or_running_bouts” (used to create the next activity encoding) with the walking or running bouts. 
The column has the following values:

“<1 min walking”

“>=1min & <5min walking”

“>=5min & <10min walking”

“>=10min walking”

“<1 min running”

“>=1min & <5min running”

“>=5min & <10min running”

“>=10min running”

“” - empty if no walking or running bout was detected

data - dataset (hip, wrist, thigh, or chest)

In [14]:
def get_walking_or_running_bouts(data):
    previous_person = data.iloc[0,:]["type"]
    previous = ""
    count = 0
    start = -1
    data["walking_running_bouts"] = ""
    bouts_col_index = data.shape[1]-1
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        activity = row["detailed_activity"].split(" ")[0]
        if activity in ["walking", "running"]:
            if previous == activity and previous_person == row["type"]:
                previous = activity
                count += 1
            elif previous == "":
                previous = activity
                count += 1
                start = i
            else:
                mins = count/60.0
                if mins < 1.0:
                    label = "<1 min " + previous
                elif mins < 5.0:
                    label = ">=1min & <5min " + previous
                elif mins < 10.0:
                    label = ">=5min & <10min " + previous
                else:
                    label = ">=10min " + previous
                for j in range(start, i):
                    data.iat[j, bouts_col_index] = label
                count = 1
                previous = activity
                start = i
        elif count > 0:
            mins = count/60.0
            if mins < 1.0:
                label = "<1 min " + previous
            elif mins < 5.0:
                label = ">=1min & <5min " + previous
            elif mins < 10.0:
                label = ">=5min & <10min " + previous
            else:
                label = ">=10min " + previous
            for j in range(start, i):
                data.iat[j, bouts_col_index] = label
            count = 0
            previous = "" 
            start = -1
        previous_person = row["type"]
    return data

# Updated Encodings Column

Description: creates a new column in the dataset called “updated_activity” (corresponds to detailed activity in our final paper) with the updated activity encodings.

data - dataset (hip, wrist, thigh, or chest)

In [15]:
def get_new_encodings(data):
    activity_encoding = []
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        walking_bouts = row["walking_running_bouts"]
        run_or_walk = walking_bouts.split(" ")
        run_or_walk = run_or_walk[len(run_or_walk)-1]
        if walking_bouts == "":
            if row["broad_activity"] == "sit/stand":
                if row["detailed_activity"] == "sit/lie":
                    activity_encoding.append("sit/lie")
                else:
                    activity_encoding.append("stand still")
            else:
                activity_encoding.append(row["broad_activity"])
        elif run_or_walk == "running":
            activity_encoding.append("running")
        else:
            if walking_bouts == "<1 min walking":
                activity_encoding.append("mixed-activity")
            else:
                activity_encoding.append("walking")
    data["updated_activity"] = activity_encoding
    return data

# Final Activity Encoding Columns

Description: creates a new column in the dataset called “final_activity” (corresponds to detailed activity in our final paper) with the final activity encodings.

data - dataset (hip, wrist, thigh, or chest)

In [16]:
def get_final_encodings(data):
    activity_encoding = []
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        activity = row["updated_activity"]
        intensity = row["primary_intensity"]
        if activity not in ["mixed-activity", "stand still"]:
            activity_encoding.append(activity)
        elif intensity == 'light':
            activity_encoding.append("stand and move light")
        else:
            activity_encoding.append("stand and move moderate or vigorous")
    data["final_activity"] = activity_encoding
    return data

# Updated Final Activity Encoding

Description: creates a new column in the dataset called “updated_final_activity” (corresponds to final activity in our final paper) with the updated final activity encodings.

data - dataset (hip, wrist, thigh, or chest)

In [17]:
def get_updated_final_encodings(data):
    activity_encoding = []
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        activity = row["final_activity"]
        if activity in ["stand and move light", "stand and move moderate or vigorous"]:
            activity_encoding.append("stand and move")
        elif activity in ["sit/lie", "vehicle"]:
            activity_encoding.append("sit/lie")
        else:
            activity_encoding.append(activity)
        
    data["updated_final_activity"] = activity_encoding
    return data

### Hip

In [15]:
# hip = get_broad_activity_type(hip)

In [16]:
# hip = get_detailed_activity_type(hip)

In [17]:
# hip = get_walking_or_running_bouts(hip)

In [18]:
# hip = get_new_encodings(hip)

In [19]:
# hip.to_csv("Hip.csv",index=False)

In [20]:
# hip = get_final_encodings(hip)

In [21]:
# hip = get_updated_final_encodings(hip)

In [18]:
# hip.groupby(['updated_final_activity','final_activity', 'updated_activity', 'primary_intensity']).count()

In [19]:
# hip.groupby(['updated_final_activity']).count()

In [20]:
# hip.to_csv("Final_Data/Hip.csv", index = False)

### Wrist

In [21]:
# wrist = get_broad_activity_type(wrist)

In [22]:
# wrist = get_detailed_activity_type(wrist)

In [23]:
# wrist = get_walking_or_running_bouts(wrist)

In [24]:
# wrist = get_new_encodings(wrist)

In [25]:
# wrist.to_csv("Wrist.csv",index=False)

In [26]:
# wrist = get_final_encodings(wrist)

In [27]:
# wrist = get_updated_final_encodings(wrist)

In [28]:
# wrist.groupby(['updated_final_activity','final_activity', 'updated_activity', 'primary_intensity']).count()

In [29]:
# wrist.to_csv("Final_Data/Wrist.csv", index = False)

### Thigh

In [30]:
# thigh = get_broad_activity_type(thigh)

In [31]:
# thigh = get_detailed_activity_type(thigh)

In [32]:
# thigh = get_walking_or_running_bouts(thigh)

In [33]:
# thigh = get_new_encodings(thigh)

In [34]:
# thigh.to_csv("Thigh.csv",index=False)

In [35]:
# thigh = get_final_encodings(thigh)

In [36]:
# thigh = get_updated_final_encodings(thigh)

In [37]:
# thigh.groupby(['updated_final_activity','final_activity', 'updated_activity', 'primary_intensity']).count()

In [38]:
# thigh.to_csv("Final_Data/Thigh.csv", index = False)

### Chest

In [39]:
# chest = get_broad_activity_type(chest)

In [40]:
# chest = get_detailed_activity_type(chest)

In [41]:
# chest = get_walking_or_running_bouts(chest)

In [42]:
# chest = get_new_encodings(chest)

In [43]:
# chest.to_csv("Chest.csv",index=False)

In [44]:
# chest = get_final_encodings(chest)

In [45]:
# chest = get_updated_final_encodings(chest)

In [46]:
# chest.groupby(['updated_final_activity','final_activity', 'updated_activity', 'primary_intensity']).count()

In [47]:
# chest.to_csv("Final_Data/Chest.csv", index = False)